# Flood-fill algorithm

This is quite similar to the Poisson disc sampler, except it should be much less expensive. We can do away with searching within a specified radius and instead sample all neighbouring nodes. This avoids the costly neighbour cache construction.

It takes a 3 dimensional grid of zeros that has watertight surfaces marked as nonzeros. It fills up the area between surfaces from specified seed points and assigns these regions an index.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

from scipy.spatial import cKDTree
from scipy.interpolate import RegularGridInterpolator

import conduction

In [ ]:
directory2 = '/mnt/home_geo/mguerri/Documents/itherc/fullea_et_al_2014_0/'

layer_attributes = np.loadtxt(directory2+'layers.info', skiprows=1, usecols=(2,3,4,5,6,7,8,9,10))
layer_number = np.loadtxt(directory2+'layers.info', dtype=int, skiprows=1, usecols=(0,))

layer_header = ['body number', 'density', 'alpha', 'thermal conductivity', 'heat production rate',\
                'pressure coefficient', 'Gruneisen parameter', 'pressure derivative of bulk modulus', 'man']


layer = dict()
for i in xrange(0, 10):
    data = 1e3*np.loadtxt(directory2+'layers_xy/layer{}.xyz'.format(i))
    data[:,2] *= -1
    layer[i] = data

minX, minY, minZ = data[:,:3].min(axis=0)
maxX, maxY, maxZ = data[:,:3].max(axis=0)    

Xcoords = np.unique(data[:,0])
Ycoords = np.unique(data[:,1])

nx, ny = Xcoords.size, Ycoords.size
nz = 200

In [ ]:
fig = plt.figure(figsize=(15,10))
ax1 = fig.add_subplot(111, projection='3d')
ax1.scatter3D(layer[1][:,0], layer[1][:,1], layer[1][:,2])
ax1.scatter3D(layer[5][:,0], layer[5][:,1], layer[5][:,2])

minZ, maxZ = ax1.get_zlim()

**These layers are NOT watertight!**

The z-direction has a lot of holes.

In [ ]:
from scipy.interpolate import SmoothBivariateSpline, RectBivariateSpline

spl = dict()

for i in xrange(10):
    data = layer[i]
    xl = data[:,0]
    yl = data[:,1]
    zl = data[:,2].reshape(nx,ny)
    spl[i] = RectBivariateSpline(Xcoords, Ycoords, zl)

In [ ]:
mesh = conduction.Conduction3D((minX, minY, minZ), (maxX, maxY, maxZ), (nx, ny, nz))

In [ ]:
xq, yq = np.meshgrid(Xcoords, Ycoords)
xq = xq.ravel()
yq = yq.ravel()

horizontal_slice = np.column_stack([xq, yq])

# create layer voxel
layer_voxel = np.zeros((nz, ny, nx), dtype=np.int8)
layer_mask = np.zeros(nx*ny*nz, dtype=bool)

# create KDTree
tree = cKDTree(mesh.coords)

In [ ]:
layer_voxel.fill(-1)

def fill_layer(l, layer_voxel):
    layer_mask.fill(0)
    
    zq = spl[l].ev(xq, yq)
    d, idx = tree.query(np.column_stack([xq, yq, zq]))
    layer_mask[idx] = True
    
    i0, j0, k0 = np.where(layer_mask.reshape(nz,ny,nx))
    layer_voxel[i0, j0, k0] = l


fill_layer(1, layer_voxel)
fill_layer(3, layer_voxel)

In [ ]:
plt.imshow(layer_voxel[:,5,:], origin='lower')